In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
import matplotlib.pylab as plt

In [ ]:
%matplotlib inline

In [ ]:
from matplotlib.pylab import rcParams

In [ ]:
rcParams['figure.figsize']= 15, 6

In [ ]:
data = pd.read_csv('AirPassengers.csv')

In [ ]:
print data.head()

In [ ]:
dateparse = lambda dates: pd.datetime.strptime(dates, '%Y-%m')

In [ ]:
data = pd.read_csv('AirPassengers.csv',parse_dates=True, index_col='Month', date_parser=dateparse)

In [ ]:
print data.head()

In [ ]:
data.index

In [ ]:
from datetime import datetime

In [ ]:
ts = data['#Passengers']

In [ ]:
ts.head(10)

In [ ]:
ts['1949-01-01']

In [ ]:
ts[datetime(1949,1,1)]

In [ ]:
plt.plot(ts)

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
def test_stationarity(timeseries):
    
    #Determing rolling statistics
    rolmean = pd.rolling_mean(timeseries, window=12)
    rolstd = pd.rolling_std(timeseries, window=12)

    #Plot rolling statistics:
    orig = plt.plot(timeseries, color='blue',label='Original')
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    std = plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show(block=False)
    
    #Perform Dickey-Fuller test:
    print 'Results of Dickey-Fuller Test:'
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print dfoutput

In [ ]:
test_stationarity(ts)

In [ ]:
ts_log = np.log(ts)

In [ ]:
plt.plot(ts_log)

In [ ]:
moving_avg = ts_log.rolling(window=12, center=False).mean()

In [ ]:
plt.plot(moving_avg)
plt.plot(ts_log, color='red')

In [ ]:
ts_log_moving_avg_diff = ts_log - moving_avg

In [ ]:
ts_log_moving_avg_diff.dropna(inplace=True)

In [ ]:
test_stationarity(ts_log_moving_avg_diff)

In [ ]:
from statsmodels.tsa.stattools import acf, pacf

In [ ]:
ts_log_diff = ts_log - ts_log.shift()

In [ ]:
ts_log_diff.dropna(inplace=True)

In [ ]:
lag_acf = acf(ts_log_diff, nlags=20)

In [ ]:
lag_pacf = pacf(ts_log_diff, nlags=20, method='ols')

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

In [ ]:
model = ARIMA(ts_log, order=(2,1,0))

In [ ]:
results_AR = model.fit(disp=-1)

In [ ]:
plt.plot(ts_log_diff)
plt.plot(results_AR.fittedvalues, color='red')


In [ ]:
model = ARIMA(ts_log, order=(2, 1, 2))  
results_ARIMA = model.fit(disp=-1)  
plt.plot(ts_log_diff)
plt.plot(results_ARIMA.fittedvalues, color='red')
plt.title('RSS: %.4f'% sum((results_ARIMA.fittedvalues-ts_log_diff)**2))

In [ ]:
predictions = pd.Series(results_ARIMA.fittedvalues, copy=True)
print predictions.head()

In [ ]:
predictions_cumsum = predictions.cumsum()
print predictions_cumsum.head()

In [ ]:
predictions_log = pd.Series(ts_log.ix[0], index=ts_log.index)
predictions_log = predictions_log.add(predictions_cumsum,fill_value=0)
predictions_log.head()

In [ ]:
predictions_ARIMA = np.exp(predictions_log)
plt.plot(ts)
plt.plot(predictions_ARIMA)
plt.title('RMSE: %.4f'% np.sqrt(sum((predictions_ARIMA-ts)**2)/len(ts)))